In [1]:
import sys
from enum import Enum

# =========================
# Společné: pohyby + styly
# =========================

class Movement(Enum):
    NONE = 0
    LEFT = -1
    RIGHT = 1

# =========================
# Abeceda
# =========================

class Alphabet(list):
    def __init__(self, iterable=(), empty_char='#'):
        super().__init__(iterable)
        self._empty_char = empty_char
        self.append(self._empty_char)

    def __contains__(self, item: str):
        return super().__contains__(item)

    def append(self, value: str):
        if not super().__contains__(value):
            super().append(value)

    def extend(self, iterable):
        for item in iterable:
            if not super().__contains__(item):
                super().append(item)

    @property
    def empty_char(self) -> str:
        return self._empty_char


# =========================
# Stavy
# =========================

class StartingStateError(Exception):
    def __init__(self, count):
        if count == 0:
            super().__init__("Chybí počáteční stav stroje")
        else:
            super().__init__(f"Více než 1 ({count}) počátečních stavů stroje")


class EndingStateError(Exception):
    def __init__(self):
        super().__init__("Chybí koncový stav stroje")


class State:
    def __init__(self, state_id: int, starting: bool, ending: bool):
        self._id = state_id
        self._starting = starting
        self._ending = ending

    @property
    def id(self) -> int:
        return self._id

    @property
    def starting(self) -> bool:
        return self._starting

    @property
    def ending(self) -> bool:
        return self._ending


class States(dict[int, State]):
    def get_starting(self) -> int:
        ss = [s.id for s in self.values() if s.starting]
        if len(ss) != 1:
            raise StartingStateError(len(ss))
        return ss[0]

    def get_ending(self):
        es = [s.id for s in self.values() if s.ending]
        if len(es) == 0:
            raise EndingStateError()
        return es


In [2]:
# =========================
# Pásky
# =========================

class OutOfRangeError(Exception):
    def __init__(self, position: int):
        if position < 0:
            message = "Začátek pásky již dosažen, nelze jít vlevo"
        else:
            message = "Konec pásky již dosažen, nelze jít vpravo"
        super().__init__(message)


class Tape(list[str]):
    def __init__(self, iterable: str, _input=False, _output=False, empty_char="#"):
        super().__init__(iterable)
        self._input = _input
        self._output = _output
        self._empty = empty_char

        if self._input:
            # najdi první neprázdný symbol
            self._position = 0
            for i, ch in enumerate(self):
                if ch != self._empty:
                    self._position = i
                    break
        else:
            self._position = 0

    @property
    def input(self) -> bool:
        return self._input

    @property
    def output(self) -> bool:
        return self._output

    @property
    def value(self) -> str:
        return self[self._position]

    @value.setter
    def value(self, val: str):
        self[self._position] = val

    def move_head(self, movement: Movement):
        self._position += movement.value
        if self._position not in range(0, len(self)):
            raise OutOfRangeError(self._position)

    def resize(self, new_size: int, empty_char: str):
        if new_size <= len(self):
            return
        self.extend([empty_char] * (new_size - len(self)))

    def __str__(self):
        s = "".join(self)
        pointer = " " * self._position + "↑"
        return s + "\n" + pointer

class Tapes(list[Tape]):
    Values = tuple[str, ...]
    Movements = tuple[Movement, ...]

    @property
    def tape_values(self) -> Values:
        return tuple(t.value for t in self)

    @tape_values.setter
    def tape_values(self, values: Values):
        for i in range(len(self)):
            self[i].value = values[i]

    def move_heads(self, movements: Movements):
        for i in range(len(self)):
            self[i].move_head(movements[i])

    def resize_tapes(self, empty_char: str):
        max_length = len(max(self, key=lambda t: len(t)))
        for tape in self:
            tape.resize(max_length, empty_char)

    def __str__(self):
        return "\n".join(str(t) for t in self)


In [3]:
# =========================
# Přechody
# =========================

class TransitionKeyError(Exception):
    pass

class Transition:
    Key = tuple[int, Tapes.Values]
    Value = tuple[int, Tapes.Values, Tapes.Movements]

    def __init__(self, current_state: int, current_values: Tapes.Values,
                 new_state: int, new_values: Tapes.Values, movements: Tapes.Movements):
        self._current_state = current_state
        self._current_values = current_values
        self._new_state = new_state
        self._new_values = new_values
        self._movements = movements

    @property
    def key(self) -> Key:
        return self._current_state, self._current_values

    @property
    def value(self) -> Value:
        return self._new_state, self._new_values, self._movements

    @property
    def current_state(self) -> int:
        return self._current_state

    @property
    def current_values(self) -> Tapes.Values:
        return self._current_values

    @property
    def new_state(self) -> int:
        return self._new_state

    @property
    def new_values(self) -> Tapes.Values:
        return self._new_values

    @property
    def movements(self) -> Tapes.Movements:
        return self._movements

    def __str__(self):
        s = f"δ({self._current_state}"
        for v in self._current_values:
            s += f", {v}"
        s += f") = ({self._new_state}"
        for v in self._new_values:
            s += f", {v}"
        for m in self._movements:
            s += f", {m.name}"
        s += ")"
        return s


class Transitions(dict[Transition.Key, Transition]):
    def append(self, transition: Transition):
        self[transition.key] = transition

    def get_transition(self, key: Transition.Key) -> Transition | None:
        try:
            return self[key]
        except KeyError:
            return None


In [4]:
# =========================
# Stroj
# =========================

class TransitionStateError(Exception):
    def __init__(self, state_ids: list[int]):
        super().__init__(f"Stavy {state_ids} nejsou v množině stavů")


class TransitionValuesError(Exception):
    def __init__(self, incorrect: list[str]):
        super().__init__(f"Hodnoty z přechodu {incorrect} nejsou v abecedě")


class ContentOutOfAlphabetError(Exception):
    def __init__(self, tape_index: int, invalid: list[str]):
        super().__init__(f"Páska {tape_index} obsahuje znaky mimo abecedu:\n{invalid}")


class TransitionNotFound(Exception):
    def __init__(self, state: int, values: Tapes.Values):
        super().__init__(f"Přechod nenalezen pro stav {state} a hodnoty {values}")


class EndingStateReached(Exception):
    pass

def print_tape_with_head(tape: Tape):
    s = "".join(tape)
    pointer = " " * tape._position + "↑"
    print(s)
    print(pointer)

steps_log = []


class TuringMachine:
    def __init__(self, states: States, tapes: Tapes, transitions: Transitions, alphabet: Alphabet, empty_char: str):
        self._empty_char = empty_char
        self._alphabet = alphabet
        self._states = states
        self._transitions = transitions
        self._tapes = tapes
        self._state = states.get_starting()
        self._validate_transitions()
        self._validate_tapes()

    def __str__(self):
        return f"Stav: {self._state}\n{self._tapes}"

    def _validate_transitions(self):
        unknown = []
        for t in self._transitions.values():
            if t.current_state not in self._states or t.new_state not in self._states:
                unknown.append(t.current_state if t.current_state not in self._states else t.new_state)
        if unknown:
            raise TransitionStateError(unknown)
        bad_values = []
        a = set(self._alphabet)
        for t in self._transitions.values():
            if set(t.current_values) - a or set(t.new_values) - a:
                bad_values.extend(list(set(t.current_values) - a))
                bad_values.extend(list(set(t.new_values) - a))
        if bad_values:
            raise TransitionValuesError(bad_values)

    def _validate_tapes(self):
        for i, tape in enumerate(self._tapes):
            invalid = [ch for ch in tape if ch not in self._alphabet]
            if invalid:
                raise ContentOutOfAlphabetError(i, invalid)

    def _get_transition_key(self) -> Transition.Key:
        return self._state, self._tapes.tape_values

    def _get_transition(self) -> Transition | None:
        return self._transitions.get_transition(self._get_transition_key())

    def _write_info(self, transition: Transition):
        steps_log.append({
            "step": len(steps_log),
            "state": self._state,
            "read": self._tapes.tape_values,
            "transition": str(transition),
            "tape1": "".join(self._tapes[0]),
            "tape2": "".join(self._tapes[1]),
            "head1": self._tapes[0]._position,
            "head2": self._tapes[1]._position
        })
        print(f"Krok {len(steps_log)-1}, Stav: {self._state}")
        print("Páska 1:")
        print_tape_with_head(self._tapes[0])
        print("Páska 2:")
        print_tape_with_head(self._tapes[1])
        print(transition)
        print("-----------------------------------")

    def _iterate(self):
        tr = self._get_transition()
        if tr is None:
            raise TransitionNotFound(self._state, self._tapes.tape_values)
        self._write_info(tr)

        self._tapes.tape_values = tr.new_values
        self._tapes.move_heads(tr.movements)
        self._state = tr.new_state

        if self._states[self._state].ending:
            raise EndingStateReached
        self._iterate()

    def iterate(self):
        try:
            self._tapes.resize_tapes(empty_char=self._empty_char)
            print("Vstup:")
            print_tape_with_head(self._tapes[0])
            print_tape_with_head(self._tapes[1])
            print()
            self._iterate()
        except EndingStateReached:
            print("\nVýstup:")
            print_tape_with_head(self._tapes[1])

            print("\nKódování Turingova stroje:")
            coder = TuringMachineCoder()
            code = coder.encode(
                states=self._states,
                alphabet=self._alphabet,
                empty_char=self._empty_char,
                transitions=self._transitions
            )
            print(code)

            print("\nTabulka kroků:")
            for row in steps_log:
                print(f"Krok {row['step']}, Stav: {row['state']}, Čteno: {row['read']}")
                print(f"  Přechod: {row['transition']}")
                print()

            output_str = "".join(self._tapes[1]).strip("#")
            print("\nBinární výsledek:", output_str)
        except Exception as ex:
            print(ex)


In [5]:
# =========================
# Kódér (unární)
# =========================

class TuringMachineCoder:
    def __init__(self):
        self._begin_end = '111'
        self._separator = '11'
        self._transition_separator = '1'
        self._value = '0'

    def encode(self, states: States, alphabet: Alphabet, empty_char: str, transitions: Transitions) -> str:
        self._states = states
        self._alphabet = alphabet
        self._transitions = transitions
        self._empty_char = empty_char
        ret = ""
        ret += self._begin_end
        ret += self.encode_alphabet()
        ret += self._separator
        ret += self.encode_states()
        ret += self._separator
        ret += self.encode_transitions()
        ret += self._begin_end
        return ret

    def encode_states(self) -> str:
        ret = self._zeros(len(self._states), skip_sep=True)
        ret += self._separator
        ret += self._zeros(self._states.get_starting() + 1)
        ret += self._transition_separator
        endings = self._states.get_ending()
        for i, st in enumerate(endings):
            ret += self._zeros(st + 1, skip_sep=(i == len(endings) - 1))
        return ret

    def encode_alphabet(self) -> str:
        return self._zeros(len(self._alphabet), skip_sep=True)

    def encode_transitions(self) -> str:
        keys = list(self._transitions.keys())
        encoded = []
        for k in keys:
            encoded.append(self.encode_transition(self._transitions.get_transition(k)))
        return self._transition_separator.join(encoded)

    def encode_transition(self, t: Transition) -> str:
        ret = ""
        ret += self._zeros(t.current_state + 1)
        for v in t.current_values:
            ret += self._zeros(self._alphabet.index(v) + 1)
        ret += self._zeros(t.new_state + 1)
        for v in t.new_values:
            ret += self._zeros(self._alphabet.index(v) + 1)
        for i, m in enumerate(t.movements):
            if m == Movement.LEFT:
                mv = 1
            elif m == Movement.RIGHT:
                mv = 2
            else:
                mv = 3
            ret += self._zeros(mv, skip_sep=(i == len(t.movements) - 1))
        return ret

    def _zeros(self, cnt: int, skip_sep=False) -> str:
        s = '0' * cnt
        if not skip_sep:
            s += self._transition_separator
        return s


In [6]:
# =========================
# Konfigurace: fun(x1,..,xn)
# Struktura: 2-páskový TS,
# =========================

class FunConfig:
    def __init__(self, _input: str):

        self._raw_input = _input
        self._empty = "#"

        self._alphabet = Alphabet("01", empty_char=self._empty)

        self._states = self._build_states()
        self._tapes = self._build_tapes()
        self._transitions = self._build_transitions()

    def __str__(self) -> str:
        return TuringMachineCoder().encode(
            states=self._states,
            alphabet=self._alphabet,
            empty_char=self._empty,
            transitions=self._transitions
        )

    def _build_states(self) -> States:
        states = States()
        states[0] = State(0, True,  False)
        states[1] = State(1, False, False)
        states[2] = State(2, False, False)
        states[3] = State(3, False, False)
        states[4] = State(4, False, False)
        states[5] = State(5, False, False)
        states[6] = State(6, False, False)
        states[7] = State(7, False, False)
        states[8] = State(8, False, False)
        states[9] = State(9, False, True)
        return states

    def _build_tapes(self) -> Tapes:
        tapes = Tapes()
        E = self._empty

        tape1_content = list(E + self._raw_input + E)
        tapes.append(Tape(tape1_content, _input=True))

        out_len = max(2 * len(tape1_content), 32)
        tape2_content = list(E * out_len)
        tapes.append(Tape(tape2_content, _output=True))

        return tapes

    def _build_transitions(self) -> Transitions:
        tr = Transitions()
        E = self._empty

        # Stav 0: čtení vstupu, přesun po první pásce, druhá zatím '#'
        tr.append(Transition(0, ('0', E), 1, ('0', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(0, ('1', E), 1, ('1', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(0, (E, E), 0, (E, E), (Movement.RIGHT, Movement.RIGHT)))

        # Stav 1: pokračuje ve čtení prvního čísla
        tr.append(Transition(1, ('0', E), 1, ('0', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(1, ('1', E), 1, ('1', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(1, (E, E), 2, (E, E), (Movement.RIGHT, Movement.RIGHT)))

        # Stav 2: něco jako přechod mezi čísly / začátek další operace
        tr.append(Transition(2, ('0', E), 1, ('0', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(2, ('1', E), 1, ('1', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(2, (E, E), 3, (E, E), (Movement.LEFT, Movement.LEFT)))

        # Stav 3: návrat doleva
        tr.append(Transition(3, (E, E), 4, (E, E), (Movement.LEFT, Movement.LEFT)))

        # Stav 4: kopírování z pásky 1 na pásku 2 směrem doleva
        tr.append(Transition(4, ('0', E), 4, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(4, ('1', E), 4, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(4, (E, E), 5, (E, E), (Movement.NONE, Movement.RIGHT)))

        # Stav 5: přesun po pásce 2 doprava po zapsaných bitech
        tr.append(Transition(5, (E, '0'), 5, (E, '0'), (Movement.NONE, Movement.RIGHT)))
        tr.append(Transition(5, (E, '1'), 5, (E, '1'), (Movement.NONE, Movement.RIGHT)))
        tr.append(Transition(5, (E, E), 8, (E, E), (Movement.LEFT, Movement.LEFT)))

        # Stav 6: jedna větev aritmetiky (pravděpodobně část sčítání/odčítání)
        tr.append(Transition(6, ('0', '0'), 6, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, ('0', '1'), 6, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, ('0', E),   6, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, ('1', '0'), 6, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, ('1', '1'), 7, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, ('1', E),   6, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, (E, '0'),   8, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, (E, '1'),   8, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(6, (E, E),     5, (E, E),   (Movement.NONE, Movement.RIGHT)))

        # Stav 7: druhá větev aritmetiky (carry / přenos)
        tr.append(Transition(7, ('0', '0'), 6, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(7, ('0', '1'), 7, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(7, ('1', '0'), 7, (E, '0'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(7, ('1', '1'), 7, (E, '1'), (Movement.LEFT, Movement.LEFT)))
        tr.append(Transition(7, (E, '0'),   5, (E, '1'), (Movement.NONE, Movement.RIGHT)))
        tr.append(Transition(7, (E, '1'),   7, (E, '0'), (Movement.NONE, Movement.LEFT)))
        tr.append(Transition(7, (E, E),     8, (E, '1'), (Movement.LEFT, Movement.LEFT)))

        # Stav 8: různé větve podle kombinací symbolů – část rozhodování + přechod do koncového stavu 9
        tr.append(Transition(8, ('0', E),   5, ('0', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(8, ('1', E),   5, ('1', E), (Movement.RIGHT, Movement.RIGHT)))
        tr.append(Transition(8, (E, '0'),   9, (E, '0'), (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, (E, '1'),   9, (E, '1'), (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, (E, E),     9, (E, E),   (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, ('0', '1'), 6, ('0', '1'), (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, ('1', '0'), 6, ('1', '0'), (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, ('0', '0'), 6, ('0', '0'), (Movement.NONE, Movement.NONE)))
        tr.append(Transition(8, ('1', '1'), 6, ('1', '1'), (Movement.NONE, Movement.NONE)))

        return tr

    def get_tapes(self) -> Tapes:
        return self._tapes

    def get_transitions(self) -> Transitions:
        return self._transitions

    def get_states(self) -> States:
        return self._states

    def get_alphabet(self) -> Alphabet:
        return self._alphabet

    def get_empty_char(self) -> str:
        return self._empty


In [7]:
# =========================
# Pomocná funkce pro součin
# =========================
def preprocess_product_input(raw_input: str) -> str:
    parts = raw_input.strip("#").split("#")
    if len(parts) != 2:
        raise ValueError("Součin zatím podporuje pouze 2 čísla. Zadaný vstup má {} částí.".format(len(parts)))
    factor = parts[0]
    multiplier = int(parts[1], 2)
    return "##" + "#".join([factor] * multiplier) + "##"


In [ ]:
# =========================
# Hlavní běh
# =========================

def main():
    mode = input("Zvol režim (sum nebo multi): ").strip().lower()
    raw_input = input("Zadej vstupní řetězec (např. ##101#11#11###): ").strip()

    if mode == "sum":
        cfg = FunConfig(raw_input)
    elif mode == "multi":
        try:
            processed = preprocess_product_input(raw_input)
            print(f"Zpracovaný vstup pro součin: {processed}")
            cfg = FunConfig(processed)
        except ValueError as ve:
            print("Chyba:", ve)
            return
    else:
        print("Neplatný režim. Použij 'sum' nebo 'product'.")
        return

    tm = TuringMachine(
        states=cfg.get_states(),
        tapes=cfg.get_tapes(),
        transitions=cfg.get_transitions(),
        alphabet=cfg.get_alphabet(),
        empty_char=cfg.get_empty_char()
    )
    tm.iterate()
if __name__ == "__main__": main()